# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [38]:
!pip install selenium


In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


nav = webdriver.Chrome()
# cotacao do dolar
nav.get('https://www.google.com/')
nav.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dolar')
nav.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)


# cotacao do euro
nav.get('https://www.google.com/')
nav.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')
nav.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# cotacao do ouro
nav.get('https://dolarhoje.com/ouro-hoje/')

cotacao_ouro = nav.find_element(
    By.XPATH, '//*[@id="nacional"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro)
nav.quit()


5.5348999999999995
6.3182543970000005
324.02


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [40]:
#Importar dados e atualizar cotacoes
import pandas as pd
table = pd.read_excel('Produtos.xlsx')
display(table)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [50]:
table.loc[table['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
table.loc[table['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
table.loc[table['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# atualizar colunas
# Preço de Compras = Preço Original * Cotação
table['Preço de Compra'] = table['Preço Original'] * table['Cotação']
# Preço de Vendas = Preço de Compra * Margem
table['Preço de Venda'] = table['Preço de Compra'] * table['Margem']
 
display(table)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda,Preço de Compras
0,Câmera Canon,999.99,Dólar,5.534900,5534.844651,1.40,7748.782511,5534.844651
1,Carro Renault,4500.00,Euro,6.318254,28432.144787,2.00,56864.289573,28432.144787
2,Notebook Dell,899.99,Dólar,5.534900,4981.354651,1.70,8468.302907,4981.354651
3,IPhone,799.00,Dólar,5.534900,4422.385100,1.70,7518.054670,4422.385100
4,Carro Fiat,3000.00,Euro,6.318254,18954.763191,1.90,36014.050063,18954.763191
5,Celular Xiaomi,480.48,Dólar,5.534900,2659.408752,2.00,5318.817504,2659.408752
6,Joia 20g,20.00,Ouro,324.020000,6480.400000,1.15,7452.460000,6480.400000



### Agora vamos exportar a nova base de preços atualizada

In [51]:
table.to_excel('Produtos Nei.xlsx', index=False)
